# Example of Hidden Markov Model

In [7]:
import numpy as np
import aesara
import pymc as pm

pm.__version__

'5.0.2'

Example from stackoverflow answer:

 - https://stackoverflow.com/a/29256808/1609514

In [33]:
from IPython.core.debugger import set_trace

def generate_timesteps(N, p_init, p_trans):

    set_trace()
    timesteps = np.empty(N+1, dtype=object)

    # A success denotes being in state 2, a failure being in state 1
    timesteps[0] = pm.Bernoulli('T0', p_init)

    for i in range(1, N):
        # probability of being in state 1 at time step `i` given time step `i-1`
        p_i = p_trans[1] * timesteps[i-1] + p_trans[0] * (1-timesteps[i-1])
        timesteps[i] = pm.Bernoulli('T%d' % i, p_i)

    return timesteps


In [35]:
with pm.Model() as markov_chain:
    timesteps = generate_timesteps(10, 0.8, [0.001, 0.5])
    model = pm.MCMC(timesteps)
    model.sample(10000) # no burn in necessary since we're sampling directly from the distribution
    [np.mean(model.trace(t).gettrace()) for t in timesteps]

> /var/folders/y3/28pc8qrx3dd36zwcys8z1rzc0000gn/T/ipykernel_85892/1522797367.py(6)generate_timesteps()
      4 
      5     set_trace()
----> 6     timesteps = np.empty(N+1, dtype=object)
      7 
      8     # A success denotes being in state 2, a failure being in state 1

ipdb> q


In [10]:
k = 10

with pm.Model() as markov_chain:
    
    transition_probs = pm.Uniform('transition_probs', lower=0, upper=1, shape = 2)
    initial_state = pm.Bernoulli('initial_state', p = 0.5)
    
    def transition(previous_state, transition_probs, old_rng):
        p = transition_probs[previous_state]
        next_rng, next_state = pm.Bernoulli.dist(p = p, rng=old_rng).owner.outputs
        return next_state, {old_rng: next_rng}

    rng = aesara.shared(np.random.default_rng())
    mc_chain, updates = aesara.scan(fn=transition,
                                  outputs_info=dict(initial = initial_state),
                                  non_sequences=[transition_probs, rng],
                                  n_steps=k)
    assert updates
    markov_chain.register_rv(mc_chain, name="mc_chain", initval="prior")

with markov_chain:
    trace = pm.sample_prior_predictive(1000, compile_kwargs=dict(updates=updates))

NotImplementedError: Cannot convert transition_probs to a tensor variable.